In [1]:
import sys
import numpy as np
np.set_printoptions(threshold=100, linewidth=200)

from scipy.stats import multinomial

from pprint import pprint
from tqdm import tqdm

In [2]:
D = 100  # 文書数
V = 10   # 語彙数
S = 5    # 補助情報の異なり数

true_K = 3 # トピック数

In [3]:
# トピック分布のハイパーパラメータを設定
true_alpha = 1
true_alpha_k = [true_alpha] * true_K

# トピック分布のパラメータを生成
true_theta_dk = np.random.dirichlet(alpha=true_alpha_k, size=D)

print("真の文書トピック分布 (D, K):")
pprint(true_theta_dk)

assert np.all(np.abs(true_theta_dk.sum(axis=1) - 1.0) < 1e-5)

真の文書トピック分布 (D, K):
array([[0.47715612, 0.42480248, 0.0980414 ],
       [0.3645711 , 0.00516574, 0.63026316],
       [0.31790992, 0.43482774, 0.24726234],
       ...,
       [0.06231124, 0.91405667, 0.02363209],
       [0.15043133, 0.77092695, 0.07864172],
       [0.66226774, 0.14294696, 0.19478531]])


In [4]:
# 単語分布のハイパーパラメータを設定
true_beta_v = 1
true_beta_v = [true_beta_v] * V

# 単語分布のパラメータを生成
true_phi_kv = np.random.dirichlet(alpha=true_beta_v, size=true_K)

print("真のトピック単語分布 (K, V):")
pprint(true_phi_kv)

assert np.all(np.abs(true_phi_kv.sum(axis=1) - 1.0) < 1e-5)

真のトピック単語分布 (K, V):
array([[0.14145115, 0.05127865, 0.13721486, 0.13788182, 0.01108607, 0.12030375, 0.10666786, 0.20661903, 0.06181732, 0.02567949],
       [0.3146547 , 0.04197229, 0.01122971, 0.00996433, 0.1000627 , 0.02793935, 0.10836364, 0.09118042, 0.03683826, 0.2577946 ],
       [0.00698908, 0.0906779 , 0.00425406, 0.00746471, 0.07192889, 0.08294173, 0.21488317, 0.00440974, 0.17474627, 0.34170444]])


In [5]:
# 補助情報分布のハイパーパラメータ
true_beta_s = 1
true_beta_s = [true_beta_s] * S

# 補助情報分布のパラメータを生成
true_phi_ks = np.random.dirichlet(alpha=true_beta_s, size=true_K)

print("真の補助情報分布 (K, S):")
pprint(true_phi_ks)

assert np.all(np.abs(true_phi_ks.sum(axis=1) - 1.0) < 1e-5)

真の補助情報分布 (K, S):
array([[0.38020759, 0.15234791, 0.00928943, 0.18295682, 0.27519825],
       [0.16808777, 0.56906787, 0.0596455 , 0.10941344, 0.09378542],
       [0.09469105, 0.07640532, 0.05935909, 0.07722108, 0.69232346]])


In [6]:
## テスト文書を生成

W = [] # 文書集合を初期化
Z = [] # トピック集合を初期化

X = [] # 補助情報集合を初期化
Y = [] # 補助情報トピック集合を初期化

N_d = [None] * D        # 各文書の単語数を初期化
N_dw = np.zeros((D, V)) # 文書ごとの各語彙の出現頻度を初期化

M_d = [None] * D        # 各文書の補助情報数を初期化
M_dx = np.zeros((D, S)) # 文書ごとの各補助情報の出現頻度を初期化

min_N_d = 100 # 各文書の単語数の上限
max_N_d = 200 # 各文書の単語数の下限

min_M_d = 5  # 各文書の単語数の上限
max_M_d = 10 # 各文書の単語数の下限

for d in tqdm(range(D)):
    # 単語数を生成
    N_d[d] = np.random.randint(low=min_N_d, high=max_N_d)
    # 各単語のトピックを初期化
    true_z_dn = [None] * N_d[d]
    # 各単語の語彙を初期化
    w_dn = [None] * N_d[d]

    # 補助情報数を生成
    M_d[d] = np.random.randint(low=min_M_d, high=max_M_d)
    # 各補助情報のトピックを初期化
    true_y_dn = [None] * M_d[d]
    # 各補助情報を初期化
    x_dn = [None] * M_d[d]

    for n in range(N_d[d]):
        # トピックを生成
        z = np.random.choice(true_K, p=true_theta_dk[d])
        true_z_dn[n] = z
        # 語彙を生成
        w = np.random.choice(V, p=true_phi_kv[z])
        w_dn[n] = w
        # 単語頻度をカウント
        N_dw[d, w] += 1

    for m in range(M_d[d]):
        # 補助情報トピックを生成
        y = np.random.choice(true_K, p=true_theta_dk[d])
        true_y_dn[m] = y
        # 補助情報を生成
        x = np.random.choice(S, p=true_phi_ks[y])
        x_dn[m] = x
        # 補助情報頻度をカウント
        M_dx[d, x] += 1

    # トピック集合を格納
    Z.append(true_z_dn)
    # 単語集合を格納
    W.append(w_dn)
    # 補助情報トピック集合を格納
    Y.append(true_y_dn)
    # 補助情報集合を格納
    X.append(x_dn)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 539.01it/s]


In [7]:
# テスト文書をファイルに出力
with open("pltm.test.txt", mode="w") as f:
    print("\n".join([" ".join([str(w) for w in words]) for words in W]), file=f)

!head -n2 pltm.test.txt

5 5 0 9 1 3 0 5 7 0 7 3 6 2 5 0 5 3 4 0 2 6 5 9 1 4 1 2 4 0 5 0 7 0 1 5 0 7 3 6 0 3 4 0 0 2 4 7 0 3 7 4 0 6 5 6 0 7 7 7 5 3 7 7 9 6 2 9 3 6 9 8 9 0 8 0 9 9 0 6 6 3 7 0 0 7 6 8 8 0 8 1 2 9 6 9 0 1 4 6 9 0 8 0 6 3 0 5 6 9 9 5 6 0 0 3 0 0 7 0 7 4 9 7 1 0 7 9 6 0 7 9 7 8 5 3 2 5 9 0 0 2 3 7 1 3 9 0 2 5 0 6 8 4 5 0 9 7 7 8 5 7 0 4 5 1 5 9 0 7 0 5 9 9 6 0 9 9 0 3 0 6 6 1 5 9 7 0 0 2 0 7 7 7 9 8
0 2 1 6 3 6 6 5 1 2 6 6 8 6 8 4 3 8 6 6 6 8 9 5 9 9 7 2 7 7 9 4 8 9 3 0 9 6 1 9 9 7 9 0 6 9 9 2 9 3 4 8 9 6 9 6 1 9 8 7 6 7 6 1 8 0 9 3 5 6 9 5 6 3 6 4 5 5 4 9 9 6 8 7 9 7 1 4 9 9 9 1 8 8 7 6 9 8 1 1 4 9 1 9 2 9 4 3 6 9 3 6 8 6 8 5 6 9 2 6 9 5 0 6 9 5 7 8 2 9 5 8 9 8 4 6 5 1 9 1 9 9 1 4 5 5 2 3 8 8 9 1 9 3 6 5 6 6 8 6 7 8 6 0 8 5 6 4 4 9 9 8 9 3 9 7 3


In [8]:
# テスト補助情報をファイルに出力
with open("pltm.test.x.txt", mode="w") as f:
    print("\n".join([" ".join([str(x) for x in xs]) for xs in X]), file=f)

!head -n2 pltm.test.x.txt

4 4 4 3 0 1 3
4 4 4 4 0 1
